# Sql

In [35]:
# Import libraries
import pandas as pd
import sqlite3

In [36]:
# Sqlite Setting
def dict_factory(cursor, row):
    d = {}
    for idx, col in enumerate(cursor.description):
        d[col[0]] = row[idx]
    return d


conn = sqlite3.connect("../data/sql.db")
conn.row_factory = dict_factory
iris_df = pd.read_csv("../data/iris.csv")
iris_df.columns = [colName.replace(".", "_").lower()
                   for colName in iris_df.columns]
iris_df.to_sql("iris", conn, if_exists="replace", index=False)
cursor = conn.cursor()

In [37]:
# Define excute sql function
def execute_sql(sql, msg, showNum=5):
    cursor.execute(sql)
    sql_df = pd.DataFrame.from_dict(cursor.fetchmany(showNum))
    if len(sql_df) > 0:
        print("{} query result, show {}>= rows".format(msg, showNum))
    else:
        print("Execute {} query".format(msg))
    return sql_df

In [38]:
# TEST
sql = """
            SELECT
                *
            FROM
                iris
      """
execute_sql(sql, "TEST")

TEST query result, show 5>= rows


,petal_length,petal_width,sepal_length,sepal_width,species
0,1.4,0.2,5.1,3.5,setosa
1,1.4,0.2,4.9,3.0,setosa
2,1.3,0.2,4.7,3.2,setosa
3,1.5,0.2,4.6,3.1,setosa
4,1.4,0.2,5.0,3.6,setosa


In [39]:
# CASE
sql = """
            SELECT
                CASE
                  WHEN species = "setosa"
                      THEN "True"
                      ELSE "False"
                END AS is_setosa,
                sepal_length,
                sepal_width
            FROM
                iris
      """
execute_sql(sql, "CASE")

CASE query result, show 5>= rows


,is_setosa,sepal_length,sepal_width
0,True,5.1,3.5
1,True,4.9,3.0
2,True,4.7,3.2
3,True,4.6,3.1
4,True,5.0,3.6


In [40]:
# DISTINCT
sql = """
            SELECT DISTINCT
                species
            FROM
                iris
      """
execute_sql(sql, "DISTINCT")

DISTINCT query result, show 5>= rows


,species
0,setosa
1,versicolor
2,virginica


In [41]:
# OFFSET
sql = """
            SELECT
                *
            FROM
                iris
            LIMIT 3 OFFSET 100
      """
execute_sql(sql, "OFFSET")

OFFSET query result, show 5>= rows


,petal_length,petal_width,sepal_length,sepal_width,species
0,6.0,2.5,6.3,3.3,virginica
1,5.1,1.9,5.8,2.7,virginica
2,5.9,2.1,7.1,3.0,virginica


In [42]:
# CROSS JOIN
sql = """
            SELECT DISTINCT
                iris.species AS "iris_species",
                iris_sub.species AS "iris_sub_species"
            FROM
                iris
            CROSS JOIN
                iris AS iris_sub
      """
execute_sql(sql, "CROSS JOIN", showNum=10)

CROSS JOIN query result, show 10>= rows


,iris_species,iris_sub_species
0,setosa,setosa
1,setosa,versicolor
2,setosa,virginica
3,versicolor,setosa
4,versicolor,versicolor
5,versicolor,virginica
6,virginica,setosa
7,virginica,versicolor
8,virginica,virginica


In [43]:
# UNION
# UNION: Remove duplicate
# UNION ALL: Do not remove duplicate
sql = """
            SELECT
                *
            FROM
                iris
            WHERE
                petal_length > 6.5
            UNION
            SELECT
                *
            FROM
                iris
            WHERE
                sepal_width > 3.6
      """
execute_sql(sql, "UNION")

UNION query result, show 5>= rows


,petal_length,petal_width,sepal_length,sepal_width,species
0,1.5,0.4,5.1,3.7,setosa
1,1.5,0.3,5.1,3.8,setosa
2,1.6,0.2,5.1,3.8,setosa
3,1.9,0.4,5.1,3.8,setosa
4,1.5,0.1,5.2,4.1,setosa


In [44]:
# EXCEPT
sql = """
            SELECT
                *
            FROM
                iris
            WHERE
                petal_length > 6.5
            EXCEPT
            SELECT
                *
            FROM
                iris
            WHERE
                sepal_width > 3.6
      """
execute_sql(sql, "EXCEPT")

EXCEPT query result, show 5>= rows


,petal_length,petal_width,sepal_length,sepal_width,species
0,6.6,2.1,7.6,3.0,virginica
1,6.9,2.3,7.7,2.6,virginica
2,6.7,2.0,7.7,2.8,virginica


In [45]:
# INTERSECT
sql = """
            SELECT
                *
            FROM
                iris
            WHERE
                petal_length > 6.5
            INTERSECT
            SELECT
                *
            FROM
                iris
            WHERE
                sepal_width > 3.6
      """
execute_sql(sql, "INTERSECT")

INTERSECT query result, show 5>= rows


,petal_length,petal_width,sepal_length,sepal_width,species
0,6.7,2.2,7.7,3.8,virginica


In [46]:
# RANK
# Sqlite does not support window function
# Window function: RANK() OVER (PARTITION BY species ORDER BY petal_length
# DESC) AS Rank
sql = """
            SELECT DISTINCT
                petal_length,
                (SELECT
                    1 + count(*)
                FROM
                    iris as iris_sub
                WHERE
                    iris_sub.petal_length > iris.petal_length
                        AND
                    iris_sub.species = iris.species) AS rank,
                species
            FROM
                iris
            WHERE
                rank <= 3
            ORDER BY
                rank, species
      """
execute_sql(sql, "RANK", showNum=10)

RANK query result, show 10>= rows


,petal_length,rank,species
0,1.9,1,setosa
1,5.1,1,versicolor
2,6.9,1,virginica
3,5.0,2,versicolor
4,6.7,2,virginica
5,1.7,3,setosa
6,4.9,3,versicolor


In [47]:
# HAVING
sql = """
            SELECT
                avg(petal_length) as petal_length,
                species
            FROM
                iris
            GROUP BY
                species
            HAVING
                avg(petal_length) > 4
      """
execute_sql(sql, "HAVING")

HAVING query result, show 5>= rows


,petal_length,species
0,4.260,versicolor
1,5.552,virginica


In [48]:
# VIEW
sql = """
            CREATE TEMP VIEW IF NOT EXISTS
                species_view
            AS
                SELECT
                    species
                FROM
                    iris;
      """
execute_sql(sql, "CREATE VIEW")

sql = """ 
            SELECT
                *
            FROM
                species_view;
      """
execute_sql(sql, "VIEW")

Execute CREATE VIEW query
VIEW query result, show 5>= rows


,species
0,setosa
1,setosa
2,setosa
3,setosa
4,setosa


In [49]:
# WITH
sql = """
            WITH
                petal (length, width, species)
            AS
                (SELECT
                    petal_length,
                    petal_width,
                    species
                FROM
                    iris)
            SELECT
                length,
                width
            FROM
                petal
      """
execute_sql(sql, "WITH")

WITH query result, show 5>= rows


,length,width
0,1.4,0.2
1,1.4,0.2
2,1.3,0.2
3,1.5,0.2
4,1.4,0.2


In [50]:
# TRANSACTION
sql = """
            BEGIN TRANSACTION
      """
execute_sql(sql, "BEGIN TRANSACTION")

sql = """
            DELETE FROM
                iris
            WHERE
                species is NULL
      """
execute_sql(sql, "DELETE")

sql = """
            COMMIT
      """
execute_sql(sql, "COMMIT TRANSACTION")

Execute BEGIN TRANSACTION query
Execute DELETE query
Execute COMMIT TRANSACTION query


""


In [51]:
# Close
cursor.close()
conn.close()